In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib3
import csv
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle #洗牌
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

In [3]:
df = pd.read_csv('student-mat.csv',sep=';')  
select_df = pd.DataFrame(df)
print(select_df)

    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
5       GP   M   16       U     LE3       T     4     3  services     other   
6       GP   M   16       U     LE3       T     2     2     other     other   
7       GP   F   17       U     GT3       A     4     4     other   teacher   
8       GP   M   15       U     LE3       A     3     2  services     other   
9       GP   M   15       U     GT3       T     3     4     other     other   
10      GP   F   15       U     GT3       T     4     4   teacher    health   
11      GP   F   15       U     GT3       T     2   

In [4]:
select_df['school'] = select_df['school'].str.replace('GP','5')
select_df['school'] = select_df['school'].str.replace('MS','10')
select_df['sex'] = select_df['sex'].str.replace('F','5')
select_df['sex'] = select_df['sex'].str.replace('M','10')
select_df['address'] = select_df['address'].str.replace('U','5')
select_df['address'] = select_df['address'].str.replace('R','10')
select_df['famsize'] = select_df['famsize'].str.replace('LE3','5')
select_df['famsize'] = select_df['famsize'].str.replace('GT3','10')
select_df['Pstatus'] = select_df['Pstatus'].str.replace('T','5')
select_df['Pstatus'] = select_df['Pstatus'].str.replace('A','10')
select_df['schoolsup'] = select_df['schoolsup'].str.replace('no','5')
select_df['schoolsup'] = select_df['schoolsup'].str.replace('yes','10')
select_df['famsup'] = select_df['famsup'].str.replace('no','5')
select_df['famsup'] = select_df['famsup'].str.replace('yes','10')
select_df['paid'] = select_df['paid'].str.replace('no','5')
select_df['paid'] = select_df['paid'].str.replace('yes','10')
select_df['activities'] = select_df['activities'].str.replace('no','5')
select_df['activities'] = select_df['activities'].str.replace('yes','10')
select_df['nursery'] = select_df['nursery'].str.replace('no','5')
select_df['nursery'] = select_df['nursery'].str.replace('yes','10')
select_df['higher'] = select_df['higher'].str.replace('no','5')
select_df['higher'] = select_df['higher'].str.replace('yes','10')
select_df['internet'] = select_df['internet'].str.replace('no','5')
select_df['internet'] = select_df['internet'].str.replace('yes','10')
select_df['romantic'] = select_df['romantic'].str.replace('no','5')
select_df['romantic'] = select_df['romantic'].str.replace('yes','10')
#print(select_df)

In [5]:
select_df = shuffle(select_df)
x = select_df.drop('G3',axis =1)[['school','sex','address','famsize','Pstatus','schoolsup','famsup','paid','activities',
                                  'nursery','higher','internet','romantic','age','Medu','Fedu','traveltime','studytime',
                                  'failures','famrel','freetime','goout','Dalc','Walc','health','absences','G1','G2']]
num_x = select_df.drop('G3',axis =1)[['age','Medu','Fedu','traveltime','studytime','failures','famrel',
                                      'freetime','goout','Dalc','Walc','health','absences','G1','G2']]
bin_x = select_df.drop('G3',axis =1)[['school','sex','address','famsize','Pstatus','schoolsup','famsup','paid','activities',
                                  'nursery','higher','internet','romantic']]
y = select_df['G3'].values.reshape(-1,1)

#print(y)

In [6]:
def build_tree(feature,target):
    tree = DecisionTreeClassifier()
    tree.fit(feature, target)
    return tree

In [7]:
def HoldoutValidation(feature,target):
    x_train,x_test,y_train,y_test=train_test_split(feature,target,random_state=123, train_size=0.7,test_size = 0.3)  
    return x_train,x_test,y_train,y_test
    #train_test_split(feature,target, train_size=0.7,test_size = 0.3)
    

x1_train,x1_test,y1_train,y1_test=HoldoutValidation(num_x,y)
#print(x_train,y_train)
tree=build_tree(x1_train,y1_train)
print(tree.score(x1_test,y1_test)) 
print(tree.predict(x1_test))

0.44537815126050423
[10  8 11  0 11 15  7  9 11  0  0 10  6 14 10  0  9 15  9 12 14  9 10 13
 13  0 11 10  9 10  0 15 11 11  8 10 15  0  8 15  0  8 12 13 10 12 11  0
 15  0 10  0 18 14  8 11  9 15 15 15 12 13 12 11 15  7 15 12 10 15 10 13
 13 12 10 16 14 19 10  5 11 12 12 10  8 15 16 12 11 11 12 12  9 11 15 14
  7 12  8 11 11  6  0 13 15  8 10 16  0 10 15  0  7 13 18 10 11  0  8]


In [8]:
def K_fold(feature,target,k=3):
    kf = KFold(n_splits=k, shuffle=True)
    tmp=0
    for train_index, test_index in kf.split(feature):
        x_train, x_test = feature.iloc[train_index], feature.iloc[test_index]
        y_train, y_test = target[train_index], target[test_index]

        tree = build_tree(x_train,y_train)
        tmp +=tree.score(x_test,y_test)
    tmp/=k
    print('Percentage of the average accurate model:',tmp)   
        
K_fold(num_x,y)

Percentage of the average accurate model: 0.39480684709692343


In [9]:
#RandomForest
def RandomForest(x,y):
    forest = RandomForestClassifier(criterion='entropy',n_estimators=100)
    forest.fit(x,y)
    return forest
    
x_train,x_test,y_train,y_test = HoldoutValidation(num_x,y)     
forest = RandomForest(x_train,y_train)
print(forest.score(x_test,y_test))

0.4369747899159664


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [10]:
#KNN
accurate_rate = [0,0,0,0,0,0,0,0,0,0]

def KNN(x,y,i): # ,test_feature,test_target
    x_train,x_test,y_train,y_test =HoldoutValidation(x,y)
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train,y_train)
    accurate_rate[i]=knn.score(x_test,y_test)
    print(knn.predict(x_test))
def Scaler(x):
    scaler = StandardScaler()
    scaler.fit(x)
    scaler_feature=scaler.transform(x)
    scaler_x = pd.DataFrame(scaler_feature,columns=x.columns)
    #print(scaler_x)
    
    return scaler_x
    

scaler_x= Scaler(num_x)
for i in range(3,10):
    KNN(scaler_x,y,i)
    print('k=',i,':',accurate_rate[i])

D:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  del sys.path[0]
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys


[10  8 10  8 10 11 11  8  8  0  6  8  6 13 11  0  0 12  9  0 15  8  9 13
 11  9  8 11 11  0 10 13 11  6  9 10 11  0  0 12  0  0 12 12  8  8 10  0
 12  8 11  7 16 13  6 12  9 11 11 11 12 12 10 11 15  7 14 11  8 12  9 11
 13 11 10 10 11 18  0  8 11 12 11  8  8 12 15 10  9 11 11 14 10 11 16 11
  6 14  7 11 11 10  6 11 10  8 11 14  0 10 11  6  6 10 16  8  6  0  8]
k= 3 : 0.21008403361344538
[10  8 10  8 10 11 10  8 12  0  0 11  6 13 11  0  0 14  9 12 15  8 13 13
 11  8  0 11 11  0 10 15 11 11  9 10 11  0  0 12  0  0 11 12  8 11 10  0
  0  6 10  8 16 13  6 10  9 11 14 11 11 12 12 11 15  7 14 11  0 12  0 11
 13 11 10 10 11 18  0  8 11 11 14  8  6 12 15 10  9  9 11 14 10 11 14 11
  6 14  7 11 11 10  6 11 10 11 11 12  0 10 10  6 10 14 16  8 11  0  0]
k= 4 : 0.2605042016806723
[10  8 10 10 10 11  9  8  8  0  0 11  6 13 13  0  0 14  9 12 15  8 10 13
  6  8  0 11 11  0 10 15 11 11  9 10 15  0  0 12  0  0 11  8  8 11 10 11
  0  6 10  8 15 11 10 10  9 12 14 11 10 12 12 11 15  7 14 11  8 19 11 11
 1

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was pass

In [11]:
x1_train,x1_test,y1_train,y1_test=HoldoutValidation(bin_x,y)
#print(x_train,y_train)
tree=build_tree(x1_train,y1_train)
print(tree.score(x1_test,y1_test)) 

0.10084033613445378


In [12]:
K_fold(num_x,y)

Percentage of the average accurate model: 0.34177654406662045


In [13]:
x_train,x_test,y_train,y_test = HoldoutValidation(bin_x,y)     
forest = RandomForest(x_train,y_train)
print(forest.score(x_test,y_test))

0.07563025210084033


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [14]:
scaler_x= Scaler(bin_x)
for i in range(3,10):
    KNN(scaler_x,y,i)
    print('k=',i,':',accurate_rate[i])

D:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  del sys.path[0]


[11 10  8 11  6  6  6  8  9  5 15 10  6  0  0  6  8 11 10  9  8  8  0 15
  6 11 10  8 10 15  7  9  8  8  9 11 10 10  0  8  0  7 12  8  6  6  0  0
  6 16 13  6  8 11 10 11  8 11  0  8 11  0 10 15 11 10  6 10  0  6  8  6
 15 18 10 12  6 13  9 13  9 15  6  0 10  7 13 10  9  8  0 15  8 10  9  6
  8 17  8 16 11  9  0 11 12  9 12  8  0  9 13  9  0  8 10  0  8  0 10]
k= 3 : 0.1092436974789916
[11 10  0 11  6  6 11  8  9  5 15 10  6 11  0  6  0 11 10 16  8  0 15 13
 16 11 10  0 10 13  7  9  6 10  8 11 10 10  0  8  0  7 12  8  6 13  0  0
  8 16  0 16  8 11 10 11  0 11  0  0 11  0 11 15 11 10  6 10  0  6  0  6
 15 18 10  5  6  0 14  0  9 15  6  0 10  9  0 16 14 11  0 13 11 10 14  6
  8 11  8 16 11  9  0 11  5  9 12  8 11  9 13  9  0  8 16  0  0  0 10]
k= 4 : 0.12605042016806722
[11  8 11 11  6 11 11 15  9  5 15  7  6 11  0  6  0 11 10 16  8  0 15 13
 16 11 10 11 10 13 10 12  6 10  8 11 10 10  0  8  0  7  8  8  6 13  0  0
  8 16  0 16 10 11 10 11  0 11 10  0 11  0 10 15 11 10  0 10  0 13  0  0
 1

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was pass